# Cuadratura con intervalos 
## Luis Eduardo Ramírez Montoya
### Temas Temas Selectos de Fı́sica Computacional III
### Facultad de Ciencias
### Universidad Nacional Autónoma de México
#### 7 de diciembre de 2022

In [4]:
using CuadraturaIntervalos

# Problema 

El objetivo es utilizar los conceptos que hemos desarrollado durante el curso para calcular rigurosamente la integral 

$$
    I = \int_a^b f(x) \ dx.
$$

En otras palabras, buscamos un intervalo $\mathcal{I}$ tal que 

$$
    I = \int_a^b f(x) \ dx \in \mathcal{I}. 
$$

# Método simple 

La forma más inmediata de calcular $I$ es dividir el dominio de integración en un montón de subintervalos

$$
    [a, b] = [x_0, x_1]\cup [x_1, x_2] \cup \cdots \cup [x_{N-1}, x_N].
$$

y sumar 

$$
    I \in \sum_{i=1}^N \text{diam}([x_{i-1}, x_i])F([x_{i-1}, x_i]),
$$

donde $\text{diam}([a, b]) = b - a$ y $F$ es la extensión intervalar de $f$. 
    
Ver la ecuación (5.14) de Tucker, W. (2011). Validated Numerics. Estados Unidos: Princeton University Press.

```julia
@doc raw"""
    cuadratura_simple(f::Function, D::Intervalo, N::Int = 100)

Calcula la integral de `f` en `D` usando la ecuación (5.14) de Tucker, W. (2011). Validated 
Numerics. Estados Unidos: Princeton University Press con `N` pasos de igual longitud. 
"""
function cuadratura_simple(f::Function, D::Intervalo, N::Int = 100)

    @assert isfinite(D) "Para intervalos con algún extremo infinito usar `cuadratura_infinita`"

    # Iniciamos la suma en 0
    quad = zero(D)
    # Paso 
    dt = diam(D)/N
    # Iteramos sobre el número de intervalos 
    for i in 1:N
        # Intervalo local 
        D_0 = Intervalo(D.infimo + (i-1)*dt, D.infimo + i*dt)
        # Intervalo ingenuo 
        quad += diam(D_0) * f(D_0)
    end

    return quad
end
```

# Método avanzado 

Para mejorar las cotas podemos expandir $f$ en una serie de Taylor de orden $n$ 

$$
    f(x) = \sum_{k = 0}^{n-1}f_k(\tilde{x})(x - \tilde{x})^k + f_n(\zeta_x)(x - \tilde{x})^n,
$$

donde $\tilde{x}\in\mathbf{x}$.
    
Podemos encerrar el residuo en la expansión anterior mediante 

$$
    \epsilon_n = \text{mag}(F_n(\mathbf{x}) - f_n(\tilde{x})),
$$

donde $\text{mag}([a, b]) = \max\lbrace |a|, |b|\rbrace$. 
    
Entonces 

$$
    f(x) \in \sum_{k = 0}^{n}f_k(\tilde{x})(x - \tilde{x})^k + [-\epsilon_n, \epsilon_n]|x - \tilde{x}|^n.
$$

A continuación, integramos 

$$
    \begin{split}
        \int_{\tilde{x}-r}^{\tilde{x}+r} f(x) \ dx & \in \int_{\tilde{x}-r}^{\tilde{x}+r}\left[\sum_{k = 0}^{n}f_k(\tilde{x})(x - \tilde{x})^k + [-\epsilon_n, \epsilon_n]|x - \tilde{x}|^n\right] \ dx \\
        & = \sum_{k=0}^n f_k(\tilde{x})\int_{-r}^r x^k \ dx + [-\epsilon_n, \epsilon_n] \int_{-r}^r |x|^n \ dx \\
        & = \sum_{k=0}^{\lfloor n/2\rfloor} f_
        {2k}(\tilde{x})\int_{-r}^r x^{2k} \ dx + [-\epsilon_n, \epsilon_n] \int_{-r}^r |x|^n \ dx \\
        & = 2\left(\sum_{k=0}^{\lfloor n/2\rfloor}f_{2k}(\tilde{x})\frac{r^{2k+1}}{2k+1} + [-\epsilon_n, \epsilon_n]\frac{r^{n+1}}{n+1}\right).
    \end{split}
$$

```julia 
@doc raw"""
    riemann_term(f::Function, D::Intervalo, order::Int)

Calcula el término de Riemman de `f` en `D` usando una serie de Taylor de orden `order`. 
Ver la ecuación (5.18) de Tucker, W. (2011). Validated Numerics. Estados Unidos: Princeton 
University Press. 
"""
function riemann_term(f::Function, D::Intervalo, order::Int)
    # Punto medio
    x = mid(D)
    # Expansión de Taylor (normal)
    f_t = taylor_expand(f, x; order = order)
    # Expansión de Taylor (intervalar)
    F_t = taylor_expand(f, D; order = order)
    # Epsilon 
    ϵ = mag( F_t[order] - f_t[order] )
    # Intervalo epsilon
    ϵ_I = Intervalo(-ϵ, ϵ)
    # Radio 
    r = rad(D)

    term = zero(D)

    # Término de Riemann 
    for k in 0:floor(Int, order/2)
        term += f_t[2*k] * r^(2*k + 1) / (2*k + 1)
    end
    term += ϵ_I * r^(order + 1) / (order + 1)

    return 2 * term
end
```

Al último renglón de la ecuación anterior le llamaremos el término de Riemann de $\mathbf{x}$

$$
    T_n(f, \mathbf{x}) = 2\left(\sum_{k=0}^{\lfloor n/2\rfloor}f_{2k}(\tilde{x})\frac{r^{2k+1}}{2k+1} + [-\epsilon_n, \epsilon_n]\frac{r^{n+1}}{n+1}\right).
$$
    
Entonces, la integral está contenida en 

$$
    \int_a^b f(x) \ dx \in \sum_{i=1}^N T_n(f, \mathbf{x}_i).
$$
    
Ver la ecuación (5.18) de Tucker, W. (2011). Validated Numerics. Estados Unidos: Princeton University Press.

```julia
@doc raw"""
    cuadratura_uniforme(f::Function, D::Intervalo, N::Int = 100, order::Int = 10)   

Calcula la integral de `f` en `D` usando la ecuación (5.18) de Tucker, W. (2011). Validated 
Numerics. Estados Unidos: Princeton University Press con `N` pasos de igual longitud. 
"""
function cuadratura_uniforme(f::Function, D::Intervalo, N::Int = 100, order::Int = 10)
    
    @assert isfinite(D) "Para intervalos con algún extremo infinito usar `cuadratura_infinita`"
    
    # Iniciamos la suma en 0
    quad = zero(D)
    # Paso 
    dt = diam(D)/N
    # Iteramos sobre el número de intervalos 
    for i in 1:N
        # Intervalo local 
        D_0 = Intervalo(D.infimo + (i-1)*dt, D.infimo + i*dt)
        # Término de Riemann
        quad += riemann_term(f, D_0, order)
    end

    return quad
    
end
```

# División del dominio 

- Uniforme:

$$
    \mathbf{x}_i = \left[a + \frac{i-1}{N}(b-a), a + \frac{i}{N}(b-a)\right].
$$
    
- Adaptativa: Bisectamos el dominio recursivamente hasta que cada término de Riemann sea menor que cierta tolerancia $\tau$.

$$
    \begin{array}{l}
        T_n(f, [a, b]) \\
        T_n(f, [a, x_1]) + T_n(f, [x_1, b]) \\
        T_n(f, [a, x_1]) + T_n(f, [x_1, x_2]) + T_n(f, [x_2, b]) \\
        \vdots
    \end{array}
$$

```julia
@doc raw"""
    cuadratura_adaptativa(f::Function, D::Intervalo, tol::T = 1e-8, order::Int = 10) where {T <: AbstractFloat}

Calcula la integral de `f` en `D` usando la ecuación (5.18) de Tucker, W. (2011). Validated 
Numerics. Estados Unidos: Princeton University Press con pasos adaptativos. 
"""
function cuadratura_adaptativa(f::Function, D::Intervalo, tol::T = 1e-8, order::Int = 10) where {T <: AbstractFloat}
    
    @assert isfinite(D) "Para intervalos con algún extremo infinito usar `cuadratura_infinita`"

    # Término de Riemann inicial 
    quad = riemann_term(f, D, order)
    
    if diam(quad) ≤ tol
        # El término de Riemann es más pequeño que tol -> terminar 
        return quad
    else 
        # Bisectar 
        left, right = bisecta(D)
        # Aplicar cuadratura adaptativa de forma recursiva 
        return cuadratura_adaptativa(f, left, tol/2, order) + cuadratura_adaptativa(f, right, tol/2, order)
    end
    
end
```

# Redondeo 

- Simple:

$$
    \mathbf{x} + \mathbf{y} = \left[\texttt{prevfloat}\left(\underline{x}, \underline{y}\right), \texttt{nextfloat}\left(\overline{x}, \overline{y}\right)\right],
$$

por ejemplo 

$$
    [0, 1] + [0, 1] = [-5.0\times10^{-324}, 2.0000000000000004].
$$
    
- Avanzado: la paquetería `RoundingEmulator.jl` permite tener cotas más ajustadas en ciertos casos 

$$
    \mathbf{x} + \mathbf{y} = \left[\texttt{add\_down}\left(\underline{x}, \underline{y}\right), \texttt{add\_up}\left(\overline{x}, \overline{y}\right)\right],
$$

por ejemplo 

$$
    [0, 1] + [0, 1] = [0.0, 2.0].
$$

In [55]:
function mostrar_integrales(f, D, N, τ, order)
    
    I_1 = cuadratura_simple(f, D, N)
    I_2 = cuadratura_uniforme(f, D, N, order)
    I_3 = cuadratura_adaptativa(f, D, τ, order)
    
    metodos = Dict(
        1 => rpad("Simple uniforme: ", 25), 
        2 => rpad("Avanzado uniforme: ", 25), 
        3 => rpad("Avanzado adaptativo: ", 25)
    )
    Is = [I_1, I_2, I_3]
    
    for i in 1:3
        println(metodos[i], rpad(Is[i], 50), "diam(I) = ", diam(Is[i]))
    end
end

mostrar_integrales (generic function with 1 method)

# Integral sencilla

$$
    \int_{-\pi/2}^{\pi/2}\sin(x) \ dx = 0.
$$

In [56]:
N = 1_000
order = 20
τ = 1e-8

f(x) = sin(x)
D = -π/2 .. π/2

[-1.5707963267948966, 1.5707963267948966]

In [57]:
mostrar_integrales(f, D, N, τ, order)

Simple uniforme:         [-0.003141592653633906, 0.003141592653632581]     diam(I) = 0.006283185307266487
Avanzado uniforme:       [-4.445445747625065e-14, 4.5811878596202504e-14]  diam(I) = 9.026633607245316e-14
Avanzado adaptativo:     [-5.142845785720951e-16, 5.142845785720951e-16]   diam(I) = 1.0285691571441902e-15


# Integral intermedia 

$$
    \int_{0}^{\pi} e^x (1 + \sin(x)) \ dx = \frac{3e^\pi - 1}{2} = 34.211038949168903...
$$

In [58]:
N = 1_000
order = 20
τ = 1e-8

f(x) = exp(x) + exp(x)*sin(x)
D = 0 .. π

[0.0, 3.1415926535897936]

In [59]:
mostrar_integrales(f, D, N, τ, order)

Simple uniforme:         [34.13242214467101, 34.28979039722667]            diam(I) = 0.15736825255565634
Avanzado uniforme:       [34.21103894916805, 34.21103894916973]            diam(I) = 1.6839862837514374e-12
Avanzado adaptativo:     [34.21103894916888, 34.21103894916894]            diam(I) = 5.684341886080802e-14


# Integral avanzada 

$$
    \int_{0}^{1} x^{-x} \ dx = \sum_{n=1}^\infty n^{-n} = 1.29128599706...
$$

In [60]:
N = 1_000
order = 15
τ = 1e-8

f(x) = x ^ (-x)
D = 0 .. 1

[0.0, 1.0]

In [61]:
mostrar_integrales(f, D, N, τ, order)

Simple uniforme:         [1.2899994602842164, ∞]                           diam(I) = Inf
Avanzado uniforme:       [-∞, ∞]                                           diam(I) = Inf
Avanzado adaptativo:     [1.291285996955213, 1.2912859971701207]           diam(I) = 2.1490764723353095e-10


# Redondeo simple vs avanzado 

$$
    \int_{1/2}^{1} 1 + x^5 - x^{10} \ dx = \frac{12,913}{22,528} = 0.57319779829545454...
$$

In [62]:
N = 1_000
order = 20
τ = 1e-8

f(x) = 1 + x*x*x*x*x - x*x*x*x*x*x*x*x*x*x
D = 1/2 .. 1

[0.5, 1.0]

In [66]:
redondeo(false)
mostrar_integrales(f, D, N, τ, order)

Simple uniforme:         [0.5727057446659098, 0.5736896313847518]          diam(I) = 0.0009838867188419753
Avanzado uniforme:       [0.5731977982954078, 0.573197798295502]           diam(I) = 9.414691248821327e-14
Avanzado adaptativo:     [0.5731977982954531, 0.573197798295456]           diam(I) = 2.886579864025407e-15


In [67]:
redondeo(true)
mostrar_integrales(f, D, N, τ, order)

Simple uniforme:         [0.5727057446659319, 0.5736896313847284]          diam(I) = 0.0009838867187964562
Avanzado uniforme:       [0.5731977982954319, 0.5731977982954779]          diam(I) = 4.596323321948148e-14
Avanzado adaptativo:     [0.5731977982954545, 0.5731977982954546]          diam(I) = 1.1102230246251565e-16


# Integrales impropias 

Los métodos que hemos presentado hasta ahora funcionan cuando $a$ y $b$ son finitos, pero ¿cómo podemos calcular integrales impropias?
    
Propuesta: supongamos que queremos acotar 

$$
    \int_a^\infty f(x) \ dx.
$$

Calculamos 

$$
    \int_a^{x_1}f(x) \ dx + \cdots + \int_{x_{n-1}}^{x_{n}} f(x) \ dx \in \mathcal{I}_1 + \cdots \mathcal{I}_n
$$

hasta que 

$$
    \text{diam}(\mathcal{I}_n) < \tau.
$$

Entonces, para calcular $\mathcal{I}$ necesitamos acotar 

$$
    \int_{x_n}^\infty f(x) \ dx.
$$
    
Para que la integral anterior converga, $f(x)$ debe tender a cero conforme $x \longrightarrow \infty$

$$
    \epsilon_n = \text{mag}(\mathcal{I}_n).
$$
    
Así

$$
    \int_a^\infty f(x) \ dx \in \mathcal{I}_1 + \cdots + \mathcal{I}_n + [-\epsilon_n, \epsilon_n].
$$

```julia
function cuadratura_mas_∞(f::Function, D::Intervalo, tol::T = 1e-8, order::Int = 10) where {T <: AbstractFloat}
    
    @assert isfinite(D.infimo) && isinf(D.supremo) "Esta función sólo funciona para intervalos de la forma `[a, ∞]`"

    # Primer intervalo 
    new_D = Intervalo(D.infimo, D.infimo + 1)
    # Primera integral 
    quad = cuadratura_adaptativa(f, new_D, tol, order)
    new_quad = quad
    # "Error" 
    ϵ = mag(new_quad)

    while ϵ > tol
        # Intervalo actual 
        new_D = Intervalo(new_D.supremo, new_D.supremo + 1)
        # Integral actual 
        new_quad = cuadratura_adaptativa(f, new_D, tol, order)
        # "Error" 
        ϵ = mag(new_quad)
        # Sumamos 
        quad += new_quad
    end

    return quad + Intervalo(-ϵ, ϵ)
end
```

In [75]:
function mostrar_integral_∞(f, D, N, τ, order)
    
    I = cuadratura_infinita(f, D, τ, order)
    println(rpad("Avanzado adaptativo: ", 25), rpad(I, 50), "diam(I) = ", diam(I))
    
end

mostrar_integral_∞ (generic function with 1 method)

$$
    \int_{0}^{\infty} \sqrt{x}e^{-x} \ dx = \frac{\pi}{2} = 0.8862269254...
$$

In [76]:
N = 1_000
order = 20
τ = 1e-8

f(x) = sqrt(x) * exp(-x)
D = 0 .. Inf

[0.0, ∞]

In [77]:
mostrar_integral_∞(f, D, N, τ, order)

Avanzado adaptativo:     [0.8862269157688264, 0.8862269280253979]          diam(I) = 1.2256571535473881e-8


$$
    \int_{0}^{\infty} x^{z-1}e^{-x} \ dx \bigg|_{z = 7/5} = \Gamma(z)\bigg|_{z=7/5} = 0.887263817503...
$$

In [78]:
N = 1_000
order = 20
τ = 1e-8

z = 7/5
f(x) = x^(z-1) * exp(-x)
D = 0 .. Inf

[0.0, ∞]

In [79]:
mostrar_integral_∞(f, D, N, τ, order)

Avanzado adaptativo:     [0.8872638101178691, 0.8872638196678068]          diam(I) = 9.549937618480442e-9
